# Inverse problem

Here we look at the 1D wave equation and solve an inverse problem. Given initial and boundary conditions for a wave equation with variable speed, find the correct speed minimizing also the values of the field a selected space-time points. This is an example in the package Sophon.jl but done to run in the Neural_PDE infrastructure.

In [ ]:
# Neural network

input_ = length(domains)
n = 16
length_vars = 2

#chain = [Chain(Dense(input_, n, σ), Dense(n, n, σ), Dense(n, 1)) for _ in 1:length_vars]
chain = [Chain(Dense(2, n, σ), Dense(n, n, σ), Dense(n, 1)), Chain(Dense(1, n, σ), Dense(n, n, σ), Dense(n, 1))]



In [ ]:



#strategy = QuadratureTraining(; abstol = 1e-6, reltol = 1e-6, batch = 200)
strategy = GridTraining(0.05)
discretization = PhysicsInformedNN(chain, strategy#, param_estim = true
, additional_loss = additional_loss
)

prob = discretize(wave, discretization)

sym_prob = symbolic_discretize(wave, discretization)

pde_inner_loss_functions = sym_prob.loss_functions.pde_loss_functions
bcs_inner_loss_functions = sym_prob.loss_functions.bc_loss_functions
#add_inner_loss_functions = sym_prob.loss_functions.add_loss_functions
loss = []
callback = function (p, l)
    push!(loss, l)
    println("loss: ", l)
    println("pde_losses: ", map(l_ -> l_(p.u), pde_inner_loss_functions))
    println("bcs_losses: ", map(l_ -> l_(p.u), bcs_inner_loss_functions))
    #println("add_losses: ", map(l_ -> l_(p.u), add_inner_loss_functions))
    return false
end


In [ ]:
# optimizer
opt = OptimizationOptimJL.BFGS()
res = Optimization.solve(prob, opt; callback, maxiters = 2000)
phi = discretization.phi

In [ ]:
ax = (title = "loss vs iterations")
fig, ax = lines(log10.(loss), label = "Loss")
axislegend()
fig

In [ ]:
#using CairoMakie

ts = range(0, 1; length=100)
xs = range(0, 1; length=100)
depvars = [:u, :c]

u_pred = [phi[1]([x, t], res.u.depvar[depvars[1]])[1] for x in xs, t in ts]
c_pred = [phi[2]([x], res.u.depvar[depvars[2]])[1] for x in xs]

u_true = [ū(x, t) for x in xs, t in ts]
c_true = 1 .+ abs2.(xs) |> vec

axis = (xlabel="x", ylabel="t", title="Analytical Solution")
fig, ax1, hm1 = heatmap(xs, ts, u_true, axis=axis; colormap=:jet)
ax2, hm2= heatmap(fig[1, end+1], xs, ts, u_pred, axis= merge(axis, (;title = "Prediction")); colormap=:jet)
ax3, hm3 = heatmap(fig[1, end+1], xs, ts, abs.(u_true .- u_pred), axis= merge(axis, (;title = "Absolute Error")); colormap=:jet)
Colorbar(fig[:, end+1], hm3)
fig

In [ ]:
fig, ax = lines(xs, c_pred, label = "Predicted c(x)")
lines!(ax, xs, c_true, label = "True c(x)", color = :red)
axislegend()
fig